Connected to .venv (Python 3.13.7)

In [64]:
import pandas as pd
import numpy as np

In [65]:
filename = "pageviews-hourly-20251001-000000.gz"

df = pd.read_csv(
    filename,
    sep=" ",  # space-separated
    header=None,  # no header in file
    names=["project", "page_title", "views", "bytes"],  # column names
    dtype={"project": str, "page_title": str, "views": int, "bytes": int},
    compression="gzip",  # read directly from .gz
    engine="c",  # faster parsing
)
print(f"Total views: {df['views'].sum():,}")

Total views: 16,115,693


In [ ]:
# --- Step 1: Keep only English articles --------------------------------------
df = df[df["project"] == "en"]
df.reset_index(drop=True)
df.insert(0, "row_number", range(1, len(df) + 1))
print(f"Filtered to {len(df)} English Wikipedia pages")
print(f"Total views: {df['views'].sum():,}")
print(df.head(10))

Filtered to 1184804 English Wikipedia pages
Total views: 3,092,228
        RowNumber project                                         page_title  \
520115          1      en                                                 !!   
520116          2      en                                                !!!   
520117          3      en                                           !DOCTYPE   
520118          4      en                                      !Xóõ_language   
520119          5      en                                              \&\""   
520120          6      en                                  \21_Azer\"_Medal"   
520121          7      en                                       \A\"_Device"   
520122          8      en                                 \A\"_Is_for_Alibi"   
520123          9      en  \Air\"_from_Johann_Sebastian_Bach's_Orchestral...   
520124         10      en                               \Awaken,_My_Love!\""   

        views  bytes  
520115      1      0  
520116

In [67]:
# Save filtered English dataset
df.to_csv("pageviews_en.csv", index=False)
print("✅ Saved English-only dataset: pageviews_en.csv")

✅ Saved English-only dataset: pageviews_en.csv


In [68]:
# --- Step 2: Sample articles ---------------------------------
sample_fraction = 0.01  # adjust as needed (e.g., 0.05 = 5%)
df_sample = df.sample(frac=sample_fraction, random_state=42)
df_sample.sort_values(by="views", ascending=False, inplace=True)
print(df_sample)
print(f"Total views in sample: {df_sample['views'].sum():,}")

         RowNumber project                         page_title  views  bytes
1069906     549792      en                        Jon_Stewart   2998      0
1273067     752953      en                          New_Relic    279      0
1073119     553005      en                      Josh_Hartnett    114      0
1554339    1034225      en            The_Odyssey_(2026_film)    110      0
1028735     508621      en           International_Coffee_Day     91      0
...            ...     ...                                ...    ...    ...
1058992     538878      en                               Jins      1      0
697793      177679      en                    Builder's_plate      1      0
671456      151342      en           Bill_and_Ben_(TV_series)      1      0
1697308    1177194      en                      Zapata,_Texas      1      0
1472791     952677      en  Spaulding_Rehabilitation_Hospital      1      0

[11848 rows x 5 columns]
Total views in sample: 30,806


In [69]:
df_sample.to_csv("pageviews_en_sample_1.csv", index=False)

In [ ]:
# --- Step 4: Create one operation per view ------------------------------------
# Efficiently repeat rows by the 'views' column
ops = df_sample.loc[
    df_sample.index.repeat(df_sample["views"]), ["row_number", "page_title"]
].reset_index(drop=True)
print(f"Generated {len(ops):,} total operations (1 per view)")

Generated 30,806 total operations (1 per view)


In [71]:
# --- Step 5: Randomly turn 5% of lookups into updates --------------------------
n_ops = len(ops)
n_updates = int(0.05 * n_ops)

rng = np.random.default_rng(seed=42)
update_indices = rng.choice(n_ops, size=n_updates, replace=False)

ops["op_type"] = "lookup"
ops.loc[update_indices, "op_type"] = "update"
print(f"Assigned {n_updates:,} updates ({100*n_updates/n_ops:.2f}%)")

Assigned 1,540 updates (5.00%)


In [72]:
# --- Step 6: Shuffle operations -----------------------------------------------
ops = ops.sample(frac=1, random_state=42).reset_index(drop=True)
ops

,RowNumber,page_title,op_type
0,936725,Sinéad_Keenan,lookup
1,1074397,Twiggy_Ramirez,lookup
2,984779,Sóstenes_Cavalcante,lookup
3,844392,Pyeonghwa_Paso_900,lookup
4,255751,Cole_Tucker,lookup
...,...,...,...
30801,567314,Karuvaki,lookup
30802,405032,Freddie_Stroma,lookup
30803,549792,Jon_Stewart,lookup
30804,1131869,WFAA,lookup


In [73]:
# --- Step 7: Change first occurrence per page_title to insert -----------------
# first_indices = ops.drop_duplicates(subset="page_title", keep="first").index
# ops.loc[first_indices, "op_type"] = "insert"
# print(f"Changed {len(first_indices):,} first accesses to inserts")

In [74]:
ops.to_csv("operations_en_sample_1.csv", index=False)
print("✅ Saved operations dataset: operations_en_sample_1.csv")
print(f"Contains {len(ops):,} operations (1 per view), with {n_updates:,} updates ({100*n_updates/n_ops:.2f}%)")

✅ Saved operations dataset: operations_en_sample_1.csv
Contains 30,806 operations (1 per view), with 1,540 updates (5.00%)


In [79]:
print((df_sample['views'] == 1).sum())

8175


In [82]:
# Compute string lengths
lengths = df_sample["page_title"].str.len()

# Calculate stats
avg_length = lengths.mean()
max_length = lengths.max()
min_length = lengths.min()
print(f"Page title lengths - Avg: {avg_length:.2f}, Max: {max_length}, Min: {min_length}")

Page title lengths - Avg: 20.50, Max: 168, Min: 1
